Final Stock Report Generator by Jason Yan, James Attard, Grace Shen, Cathy Zou, Alex Kartel

1. import modules

In [3]:
import yfinance as yf
import matplotlib.pyplot as plt
from fpdf import FPDF
import os
import pandas as pd
import stock_input


In [4]:
stock_code_input = stock_input.get_stock_code().upper()
print(stock_code_input)


CBA.AX


2. Retrieve financial data from functions made from James

In [5]:
import ratios_james

stock_financial_data = ratios_james.stock_financials(stock_code_input)

Import commentary 

In [6]:
commentary = ""
with open("example_files/commentary_CBA.txt", "r") as file:
    commentary = file.read()


Import DCF

In [7]:
import extract_dcf_excel_alex
dcf = extract_dcf_excel_alex.get_dcf_extract()
# pandas dataframe

Import price history chart

In [8]:
import stock_price_chart_grace
stock_price_chart_grace.stock_price_chart()

In [9]:
import image_cathy
# this creates commbank_logo.jpg in current directory
import table_cathy
# this creates commbank_table.csv in current directory

Image URL: https://upload.wikimedia.org/wikipedia/en/thumb/9/9c/Commonwealth_Bank_logo_2020.svg/120px-Commonwealth_Bank_logo_2020.svg.png
Image saved as commbank_logo.png
Second Table:
                        Order                        Name  \
0                           1             Sir John Garvan   
1                           2       Sir Robert Gibson GBE   
2                           3     Sir Claude Reading KCMG   
3  Board abolished, 1945–1960  Board abolished, 1945–1960   
4                           4     Sir Warren McDonald KBE   

                   Term start                    Term end  \
0             13 October 1924              30 August 1926   
1           13 September 1926              1 January 1934   
2              4 January 1934              21 August 1945   
3  Board abolished, 1945–1960  Board abolished, 1945–1960   
4              1 January 1960            12 November 1965   

               Time in office                       Notes  
0            1 year, 

Placing Everything together using fpdf

In [15]:
from fpdf import FPDF
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata

def clean_text(text):
    # Replace common problematic characters first
    text = text.replace('\u2014', '-')  # em dash → hyphen
    text = text.replace('\u2013', '-')  # en dash → hyphen
    text = text.replace('\u2018', "'").replace('\u2019', "'")  # curly quotes → straight
    text = text.replace('\u201c', '"').replace('\u201d', '"')

    # Normalize to ASCII and remove any remaining non-latin1 characters
    return unicodedata.normalize('NFKD', text).encode('latin-1', 'ignore').decode('latin-1')
chart_path = "price_chart.png"




# === PDF Class ===
class EquityReportPDF(FPDF):
    def header(self):
        self.image("commbank_logo.png", x=160, y=10, w=35)
        self.set_font("Helvetica", "B", 16)
        self.set_text_color(0, 51, 153)
        self.cell(0, 10, "JAGC", ln=1)
        self.set_font("Helvetica", "", 12)
        self.set_text_color(0, 0, 0)

    def add_title(self, company):
        self.set_font("Helvetica", "B", 20)
        self.set_text_color(0, 102, 204)
        self.ln(5)
        self.cell(0, 10, company, ln=1)
        self.set_font("Helvetica", "", 12)
        self.set_text_color(0)
        self.ln(3)

    def add_recommendation(self, text):
        self.set_font("Helvetica", "B", 18)
        self.set_text_color(0, 100, 0)

        self.multi_cell(0, 6, text)
        self.ln(2)


    def add_summary(self, text):
        self.set_font("Helvetica", "", 11)
        self.set_text_color(0, 0, 0)

        self.multi_cell(0, 6, text)
        self.ln(2)

    def add_financials(self, title, stock_financial_data):
        self.set_font("Helvetica", "B", 11)
        self.cell(0, 8, title, ln=1)
        self.set_text_color(0, 0, 0)

        self.set_font("Helvetica", "", 10)
        for key, val in stock_financial_data.items():
            self.cell(60, 8, f"{key}:", border=0)
            self.cell(0, 8, str(val), border=0, ln=1)
        self.ln(2)


    

    def add_table(self, title, df):
        self.set_font("Helvetica", "B", 11)
        self.cell(0, 8, title, ln=1)
        self.set_font("Helvetica", "B", 9)

        col_widths = [self.get_string_width(col) + 10 for col in df.columns]
        max_width = max(col_widths)
        col_widths = [max_width] * len(df.columns)

        # Header
        for i, col in enumerate(df.columns):
            self.cell(col_widths[i], 8, str(col), border=1)
        self.ln()

        # Rows
        self.set_font("Helvetica", "", 9)
        for _, row in df.iterrows():
            for i, val in enumerate(row):
                self.cell(col_widths[i], 8, str(val), border=1)
            self.ln()

    def add_image(self, image_path, title=None):
        if title:
            self.set_font("Helvetica", "B", 11)
            self.cell(0, 8, title, ln=1)
        self.image(image_path, w=180)
        self.ln(5)

# === Generate PDF ===
pdf = EquityReportPDF()
pdf.add_page()
pdf.add_title("Commonwealth Bank of Australia (CBA)")
pdf.add_recommendation("BUY")
pdf.add_financials("Key Financials Summary", stock_financial_data)

pdf.add_summary(clean_text(commentary))
pdf.add_image(chart_path, title="CBA Historical Price")
pdf.add_table("Discounted Cash Flow (DCF) Table", dcf)
# pdf.add_table("Chairman of the Board History", dcf)
df_chairs = pd.read_csv("commbank_table.csv")

df_chairs = df_chairs.map(lambda x: clean_text(str(x)))

pdf.add_table("Chairpersons of Commonwealth Bank", df_chairs)

output_path = "CBA.pdf"
pdf.output(output_path)
print(f"PDF report generated: {output_path}")


PDF report generated: CBA.pdf
